# Chatbot

Les chatbots sont extrêmement utiles pour les entreprises et les clients. La majorité des gens préfèrent parler directement à partir d'une boîte de dialogue au lieu d'appeler les centres de service. Facebook a publié des données qui ont prouvé la valeur des bots. Plus de 2 milliards de messages sont envoyés chaque mois entre les particuliers et les entreprises. L'étude HubSpot nous indique que 71% des personnes souhaitent bénéficier du support client des applications de messagerie. C'est un moyen rapide de résoudre leurs problèmes afin que les chatbots aient un bel avenir dans les organisations.

Aujourd'hui, nous allons construire un  chatbot à partir de zéro qui sera capable de comprendre de quoi parle l'utilisateur et de donner une réponse appropriée.

## Comment fonctionnent les chatbots?

Les chatbots ne sont rien d'autre qu'un logiciel intelligent qui peut interagir et communiquer avec des personnes comme les humains.

Tous les chatbots relèvent des concepts de NLP (Natural Language Processing). La NLP est composée de deux choses:

*     **NLU** (Natural Language Understanding): La capacité des machines à comprendre le langage humain comme l'anglais.

*     **NLG** (Natural Language Generation): capacité d'une machine à générer du texte similaire à des phrases écrites par des humains.

Imaginez un utilisateur posant une question à un chatbot: "Hé, qu'est-ce qu'il y a dans l'actualité aujourd'hui?"

Le chatbot décomposera la phrase de l'utilisateur en deux choses: l'intention et une entité. L'intention de cette phrase peut être get_news car elle fait référence à une action que l'utilisateur souhaite effectuer. L'entité donne des détails spécifiques sur l'intention, donc "aujourd'hui" sera l'entité. Ainsi, un modèle d'apprentissage automatique est utilisé pour reconnaître les intentions et les entités du chat.

## Let's Code

### Step 1. Importez les bibliothèques et chargez les données


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle



In [5]:
intents_file = open('data/Liste_d_intention.json', encoding='utf-8')
intents = json.load(intents_file)

In [7]:
print("questions: ",intents['intentions'][0]['questions'])
print("reponses: ",intents['intentions'][0]['réponses'])

questions:  [' Bonjour ', ' Comment allez-vous ? ', ' Salut ', ' Quoi de neuf ', 'Salut !\xa0', 'Ça va ?\xa0']
reponses:  ['comment ça va ?', 'Bonjour partenaire !', 'Bonjour', 'Bonjour !', 'Comment allez-vous ?', 'tout va bien ?', 'tout se passe bien pour vous ?', 'ça va chez vous ? ', 'Que puis-je faire pour vous aider?', 'En quoi puis-je vous être utile?']


### Step 2. Prétraitement des données

Le modèle ne peut pas prendre les données brutes. Il doit passer par beaucoup de prétraitement pour que la machine le comprenne facilement. Pour les données textuelles, il existe de nombreuses techniques de prétraitement disponibles. La première technique est la tokenisation, dans laquelle nous divisons les phrases en mots.

En observant le fichier d'intents, nous pouvons voir que chaque balise contient une liste de qeustions et de réponses. Nous tokenisons chaque question et ajoutons les mots dans une liste. De plus, nous créons une liste de classes et de documents pour ajouter toutes les intentions associées aux modèles.

In [8]:
words = []
classes = []
documents = []
#ignore_letters = ['!','?', ',', ';','.']

for intent in intents['intentions']:
    for pattern in intent['questions']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Bonjour'], 'salutations'), (['Comment', 'allez-vous', '?'], 'salutations'), (['Salut'], 'salutations'), (['Quoi', 'de', 'neuf'], 'salutations'), (['Salut', '!'], 'salutations'), (['Ça', 'va', '?'], 'salutations'), (['quelle', 'sont', 'les', 'différentes', 'pieces', 'demander', '?'], 'piecesAfournir'), (['quelle', 'sont', 'les', 'papiers', 'que', 'vous', 'demendez', '?'], 'piecesAfournir'), (['dites', 'moi', 'les', 'pieces', 'à', 'amener', 'pour', "s'inscrir", '?'], 'piecesAfournir'), (['donner', 'moi', 'les', 'papiers', 'demender', 'lors', 'de', "l'inscription", '?'], 'piecesAfournir'), (['citez', 'moi', 'les', 'pieces', 'dont', 'on', 'a', 'besoin', 'pour', "s'inscrir", '?'], 'piecesAfournir'), (['pouriez', 'vous', 'me', 'donner', 'des', 'renseignement', 'sur', 'les', 'papiers', 'à', 'fournir', '?'], 'piecesAfournir'), (['votre', 'formation', 'est', 'accée', 'sur', 'quel', 'domaine', '?'], 'domaine '), (['quels', 'sont', 'les', 'domaines', 'sur', 'lesquels', 'vous', 'dispensez', 'v

In [15]:
classes

['auRevoir',
 'biblioteque',
 'conditionsDAdmissibilites',
 'debutDesCours',
 'diplomesDelivres',
 'domaine ',
 'localisation',
 'modalite',
 'piecesAfournir',
 'remerciements',
 'salutations',
 'specialisation']

Une autre technique est la lemmatisation. Nous pouvons convertir les mots en forme de lemme afin de réduire tous les mots canoniques. Par exemple, les mots playing, player, players, played, etc. seront tous remplacés par play. De cette façon, nous pouvons réduire le nombre total de mots dans notre vocabulaire. Alors maintenant, nous lemmatisons chaque mot et supprimons les mots en double.

In [14]:
words = [lemmatizer.lemmatize(w.lower()) for w in words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print(len(documents), ' documents')
print(len(classes), ' intentions', classes)
print(len(words), ' mots distincts', words)

pickle.dump(words,open('checkpoints/words.pkl','wb'))
pickle.dump(classes,open('checkpoints/classes.pkl','wb'))

55  documents
12  intentions ['auRevoir', 'biblioteque', 'conditionsDAdmissibilites', 'debutDesCours', 'diplomesDelivres', 'domaine ', 'localisation', 'modalite', 'piecesAfournir', 'remerciements', 'salutations', 'specialisation']
138  mots distincts ['!', "'", '?', 'a', 'accée', 'adresse', 'allez-vous', 'amener', 'au', 'auquels', 'avez', 'avez-vous', 'avoir', 'besoin', 'biblioteque', 'bonjour', "c'est", 'ce', 'citez', 'combien', 'comment', 'condition', 'conditios', 'cours', 'cout', "d'admission", "d'eclairecissement", 'dans', 'de', 'debut', 'debuter', 'demander', 'demender', 'demendez', 'differente', 'differentes', 'différentes', 'diplomes', 'diplôme', 'dispensez', 'dites', 'domaine', 'domaines', 'donner', 'dont', 'délivrance', 'délivrez', 'délivré', 'démarage', 'en', 'est', 'est-ce', 'faire', 'faite', 'formation', 'forme', 'fournir', 'genre', 'geographique', 'information', 'ipd', 'je', "l'ecole", "l'inscription", "l'ipd", 'la', 'le', 'lesquels', 'localisation', 'lors', "m'informer", 

A la fin, **words** contiennent le vocabulaire de notre projet et **classes** contiennent le total des entités à classer. Pour enregistrer l'objet python dans un fichier, nous avons utilisé la méthode pickle.dump (). Ces fichiers seront utiles une fois la formation terminée et nous prédisons les discussions.

### Step 3. Créer des données d'entrainement et de test

Pour entraîner le modèle, nous convertirons chaque modèle d'entrée en nombres. Tout d'abord, nous allons lemmatiser chaque mot du motif et créer une liste de zéros de même longueur que le nombre total de mots. Nous définirons la valeur 1 uniquement sur les index contenant le mot dans les modèles. De la même manière, nous allons créer la sortie en définissant 1 sur l'entrée de classe à laquelle appartient le motif.

In [17]:
training = []
output_empty = [0]*len(classes)
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    #print(bag)    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)
training_X = list(training[:,0])
training_y = list(training[:,1])
print( training_y[0])
print(training_X[0])

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Step 4. Formation du modèle

L'architecture de notre modèle sera un réseau de neurones composé de 3 couches denses. La première couche a 128 neurones, la seconde en a 64 et la dernière couche aura les mêmes neurones que le nombre de classes. Les couches d'abandon sont introduites pour réduire le surajustement du modèle. Nous avons utilisé l'optimiseur SGD et ajusté les données pour démarrer l'apprentissage du modèle. Une fois l'apprentissage de 900 époques terminé, nous sauvegardons le modèle entraîné à l'aide de la fonction Keras model.save («chatbot_model.h5»).

In [40]:
model = Sequential()
model.add(Dense(128, input_shape = (len(training_X[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(training_y[0]), activation='softmax'))
sgd = SGD(lr = 0.01, decay=1e-6,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               17792     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 12)                780       
                                                                 
Total params: 26,828
Trainable params: 26,828
Non-trainable params: 0
_________________________________________________________________


In [60]:
hist = model.fit(np.array(training_X), np.array(training_y), epochs=50, batch_size=8,verbose=1)

Epoch 1/50
7/7 [==============================] - 0s 2ms/step - loss: 3.0200e-04 - accuracy: 1.0000
Epoch 2/50
7/7 [==============================] - 0s 2ms/step - loss: 7.5694e-05 - accuracy: 1.0000
Epoch 3/50
7/7 [==============================] - 0s 2ms/step - loss: 1.7640e-04 - accuracy: 1.0000
Epoch 4/50
7/7 [==============================] - 0s 2ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 5/50
7/7 [==============================] - 0s 2ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 6/50
7/7 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 7/50
7/7 [==============================] - 0s 2ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 8/50
7/7 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 9/50
7/7 [==============================] - 0s 2ms/step - loss: 1.2010e-04 - accuracy: 1.0000
Epoch 10/50
7/7 [==============================] - 0s 2ms/step - loss: 2.7198e-04 - accuracy: 1.0000
Epoch 11/50
7/7 [==

In [61]:
model.save('checkpoints.chatbot_model.h5', hist)


### Step 5. Interagir avec le chatbot

Notre modèle est prêt à chatter, alors créons maintenant une belle interface utilisateur graphique pour notre chatbot dans un nouveau fichier. Vous pouvez nommer le fichier comme gui_chatbot.py

Dans notre fichier GUI, nous utiliserons le module Tkinter pour construire la structure de l'application de bureau, puis nous capturerons le message de l'utilisateur et effectuerons à nouveau un prétraitement avant d'entrer le message dans notre modèle entraîné.

Le modèle prédira ensuite l'étiquette du message de l'utilisateur et nous sélectionnerons au hasard la réponse dans la liste des réponses de notre fichier d'intentions.

Voici le code source complet du fichier GUI.

In [62]:
import nltk
import pickle
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
import json
import random

lemmatizer = WordNetLemmatizer()
model = load_model('checkpoints.chatbot_model.h5')

intents = json.loads(open('data/Liste_d_intention.json', encoding='utf-8').read())
words = pickle.load(open('checkpoints/words.pkl','rb'))
classes = pickle.load(open('checkpoints/classes.pkl','rb'))

In [63]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower())  for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,word in enumerate(words):
            if words ==s:
                bag[i] = 1
    
    return (np.array(bag))          

In [64]:
def predict_classe(sentence):
    p = bag_of_words(sentence, words)
    
    res = model.predict(np.array([p]))[0]
    print(res)
    Error_th = 0.30
    results = [[i,r] for i,r in enumerate(res) if r>Error_th]
    results.sort(key=lambda x: x[1], reverse=True)
    
    return_list = []
    for  r in results:
        return_list.append({"intent": classes[r[0]], "probability":str(r[1])})
        
    return return_list

In [65]:
predict_classe("localisation")

[2.0145765e-02 1.3300865e-04 2.8489074e-05 1.2337204e-04 3.6158413e-05
 3.6961283e-05 3.4865056e-04 2.2160195e-04 3.6721209e-05 1.2594648e-01
 8.5290748e-01 3.5251720e-05]


[{'intent': 'salutations', 'probability': '0.8529075'}]

In [66]:
def getResponse(intents, intents_json):
    tag = intents[0]['intent']
    list_of_intents = intents_json['intentions']
    for i in list_of_intents:
        if (i['tag'] == tag):
            result = random.choice(i['réponses'])
            break
    return result

In [67]:



#Creating tkinter GUI
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
        ints = predict_classe(msg)
        res = getResponse(ints, intents)
        ChatBox.insert(END, "Bot: " + res + '\n\n')           
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", 
                    height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )


#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")

#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
root.mainloop()


[2.0145765e-02 1.3300865e-04 2.8489074e-05 1.2337204e-04 3.6158413e-05
 3.6961283e-05 3.4865056e-04 2.2160195e-04 3.6721209e-05 1.2594648e-01
 8.5290748e-01 3.5251720e-05]
[2.0145765e-02 1.3300865e-04 2.8489074e-05 1.2337204e-04 3.6158413e-05
 3.6961283e-05 3.4865056e-04 2.2160195e-04 3.6721209e-05 1.2594648e-01
 8.5290748e-01 3.5251720e-05]
[2.0145765e-02 1.3300865e-04 2.8489074e-05 1.2337204e-04 3.6158413e-05
 3.6961283e-05 3.4865056e-04 2.2160195e-04 3.6721209e-05 1.2594648e-01
 8.5290748e-01 3.5251720e-05]
[2.0145765e-02 1.3300865e-04 2.8489074e-05 1.2337204e-04 3.6158413e-05
 3.6961283e-05 3.4865056e-04 2.2160195e-04 3.6721209e-05 1.2594648e-01
 8.5290748e-01 3.5251720e-05]
